In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# !pip install pandas==1.4.2
# !pip install scikit-learn==1.0.2

In [3]:
!python -V

Python 3.9.17


## Homework 

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?


    16
    17
    18
    19


In [4]:
##!pip install fastparquet

In [5]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [8]:
print(f'Columns {len(df_jan.columns)}')

Columns 19


## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?


    32.59
    42.59
    52.59
    62.59


In [9]:
def compute_duration(data):
    data = data.copy()
    data['duration'] = ((data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']) 
                          / np.timedelta64(1, 'm'))
    return data

In [10]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']) / np.timedelta64(1, 'm')

In [11]:
df_jan = compute_duration(df_jan)
df_feb = compute_duration(df_feb)

In [12]:
df_jan['duration'].std()

42.594351241920904

## Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?


    90%
    92%
    95%
    98%


In [13]:
def drop_outliters(data):
    data = data[(data['duration']>=1) &
                                 (data['duration'] <=60)]
    return data

In [14]:
df_jan_cut_outliers = drop_outliters(df_jan)

df_feb_cut_outliers = drop_outliters(df_feb)


In [15]:
len(df_jan_cut_outliers) / len(df_jan)

0.9812202822125979

In [16]:
len(df_feb_cut_outliers) / len(df_feb)

0.9800944077722545

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    - Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
    - Fit a dictionary vectorizer
    - Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

    2
    155
    345
    515
    715


In [17]:
def one_hot(data, cols):
    data = data[cols].astype(str).copy()
    dicts = data.to_dict(orient='records')
    return dicts

In [18]:
categorical = ['PULocationID', 'DOLocationID']

train_dicts = one_hot(df_jan_cut_outliers, categorical)


In [19]:
dv = DictVectorizer()
matrix_train = dv.fit_transform(train_dicts)


In [20]:
matrix_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters
    Calculate the RMSE of the model on the training data

What's the RMSE on train?

    3.64
    7.64
    11.64
    16.64


In [21]:
target = 'duration'
y_train = df_jan_cut_outliers[target].values

lr = LinearRegression()
lr.fit(matrix_train, y_train)

y_pred = lr.predict(matrix_train)

mean_squared_error(y_train, y_pred, squared=False)

7.649261929201487

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

    3.81
    7.81
    11.81
    16.81


In [22]:
y_valid = df_feb_cut_outliers[target].values

valid_dicts = one_hot(df_feb_cut_outliers, categorical)
matrix_valid = dv.transform(valid_dicts)

y_pred_val = lr.predict(matrix_valid)

mean_squared_error(y_valid, y_pred_val, squared=False)

7.811819793542861